In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
url = "https://www.unigo.com/colleges/university-of-southern-california"

response = requests.get(url)
html = response.text

soup = BeautifulSoup(html, "html.parser")

In [8]:
def getOverallScore(soup):
    star_rating = soup.find('div', {'class': 'col-sm-2 star-rating'})
    percentage = star_rating.find('div', {'class': 'front-stars'})['style']
    percentage = percentage.replace('width:', '').replace('%', '').strip()
    return float(percentage) / 20 # out of 5

getOverallScore(soup)

4.36752136752135

In [13]:
def aboutSchool(soup):
    # pretty dumb way, but it works
    aboutSchoolStr = ""
    college_info_container = soup.find(
        'div', {'class': 'college-general-information-container'})
    h2_tags = college_info_container.find_all('h2')
    info_paragraphs = []

    if len(h2_tags) >= 2:
        first_h2_tag = h2_tags[0]
        second_h2_tag = h2_tags[1]

        for sibling in first_h2_tag.next_siblings:
            if sibling == second_h2_tag:
                break
            elif sibling.name == 'p':
                info_paragraphs.append(sibling.get_text())

    for p in info_paragraphs:
        aboutSchoolStr += p
    
    return aboutSchoolStr


aboutSchool(soup)


"Founded in 1880, University of Southern California. is a Private college. Located in California, which is a city setting in California, the campus itself is Urban. The campus is home to 18,794 full time undergraduate students, and 25,077 full time graduate students.The University of Southern California Academic calendar runs on a Semester basis. In the  school year the student to faculty ratio was 9:1. There are 2062 full time instructional teachers. Degrees awarded at University of Southern California include: Bachelor's Degree, Masters Degree, Post-master's certificate, Doctor's degree."

In [35]:
# helper
def get_second_number(lst):
    second_num = None
    for i in range(len(lst)):
        try:
            num = float(lst[i])
            if i != 0:
                second_num = num
                break
        except:
            pass
    return second_num

In [38]:
def seperateReviews(soup):
    reviews_anchor_tag = soup.find(
        'a', {'class': 'anchor-link', 'id': 'Reviews'})
    if reviews_anchor_tag:
        h4_tags = reviews_anchor_tag.find_all_next('h4')
        for h4 in h4_tags:
            h4_text = h4.get_text().strip()
            if h4_text.startswith('How would you rate'):
                h4_text = h4_text[len('How would you rate'):].strip()
                h4_text = h4_text[:-1].strip().capitalize()
                p_tag = h4.find_next_sibling('p')
                if p_tag:
                    p_text = p_tag.get_text().strip()
                    individualReview = get_second_number(p_text.split())

                    # housing_rating = float(p_text.split()[5])
                    # print(f"{h4_text}: {individualReview}")
                    print(f"{h4_text}: {individualReview}")
                    
                # print(h4_text)


seperateReviews(soup)


On-campus housing: 3.4
Off-campus housing: 3.0
Campus food: 3.6
Campus facilities: 4.4
Class size: 4.1
School activities: 4.4
Local services: 4.0
Academics: 4.0
